<a href="https://colab.research.google.com/github/pinkdolphin11/ESAA/blob/main/HW_1118_Recommendation_SurprisePackage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 08. 파이썬 추천 시스템 패키지 - Surprise

### Surprise 패키지 소개

In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


* 다양한 추천 알고리즘(ex. 사용자/아이템 기반 최근접 이웃 협업 필터링, SVD/SVD++/NMF 기반 잠재 요인 협업 필터링 등)을 쉽게 적용해 추천 시스템 구축 가능

* 사이킷런의 핵심 API와 유사한 API를 가짐

### Suprise를 이용한 추천 시스템 구축

추천 데이터를 학습용/테스트용으로 분리 후 SVD 행렬 분해를 통한 잠재 요인 협업 필터링 수행하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
data = Dataset.load_builtin('ml-100k')

# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 지정
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [ ]:
algo = SVD() # 알고리즘 객체 생성
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)
print('prediction type :',type(predictions),' size :',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

In [ ]:
# 3개의 Prediction 객체에서 uid, iid, est 속성 추출
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

In [ ]:
# predict 메서드를 이용한 추천 예측
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

* predict() : 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 est로 반환

* test() : 입력 데이터 세트의 모든 사용자와 아이템 아이디에 대해서 predict()를 반복적으로 수행한 결과

In [ ]:
accuracy.rmse(predictions)

### Suprise 주요 모듈 소개

* Dataset
  
  * 사용자 아이디, 아이템 아이디, 평점 데이터가 로우 레벨로 된 데이터 세트만 적용 가능, 이들을 각각 1 ~ 3번째 칼럼으로 가정해 데이터를 로딩하고 4번째 칼럼부터는 로딩을 수행하지 않음(데이터 칼럼 순서가 맞아야 함)

  * load_builtin : 내려받은 데이터(ml-100k/ml-1m)는 .surprise_data 디렉터리 밑에 저장됨

  * load_from_file : OS 파일에서 데이터 로딩

  * load_from_df : pandas dataframe에서 데이터 로딩

In [ ]:
# OS 파일 데이터를 Surprise 데이터 세트로 로딩
import pandas as pd
ratings = pd.read_csv('./ml-latest-small/ratings.csv')

# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)

In [ ]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp',sep=',',rating_scale=(0.5,5))
data = Dataset.load_from_file('./ml-latest-small/ratings_noh.csv',reader=reader)

* Reader 클래스의 주요 파라미터

  * line_format(string) : 칼럼을 순서대로 나열(입력된 문자열을 공백으로 분리해 칼럼으로 인식)

  * sep(char) : 칼럼을 분리하는 분리자(default : \t)

  * rating_scale (tuple, optional) : 평점 값의 최소 ~ 최대 평점을 설정

In [ ]:
# SVD 행렬 분해로 추천 예측

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

In [ ]:
# Pandas DataFrame에서 Suprise 데이터 세트로 로딩

import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점의 순서
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

### Suprise 추천 알고리즘 클래스

* SVD : 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘

* KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘

* BaselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

* 비용함수 : 사용자 베이스라인 편향성을 감안한 평점 예측에 Regularization을 적용

* SVD 클래스 입력 파라미터

  * n_factors : 잠재 요인 K의 개수(커질수록 정확도와 과적합 가능성이 높아짐)

  * n_epochs : SGD 수행 시 반복 횟수

  * based : 베이스라인 사용자 편향 적용 여부


### 베이스라인 평점

* 개인의 성향을 반ㅇ영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것

* 전체 평균 평점(모든 사용자의 아이템에 대한 평점을 평균한 값) + 사용자 편향 점수(사용자별 아이템 평점 평균 값 - 전체 평균 평점) + 아이템 편향 점수(아이템별 평점 평균 값 - 전체 평균 평점)

### 교차 검증과 하이퍼 파라미터 튜닝

In [ ]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('./ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale = (0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

algo = SVD(random_state=0)
cross_validate(algo,data,measures=['RMSE','MAE'],cv=5,verbose=False)

폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여줌

In [ ]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs':[20,40,60], 'n_factors':[50,100,200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures = ['rmse','mse'],cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

### Surprise를 이용한 개인화 영화 추천 시스템 구축

In [ ]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp',sep=',',rating_scale=(0.5,5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratingws_noh.csv',reader=reader)

# 전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [ ]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [ ]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')

# userId = 9의 movieId 데이터를 추출해 movieId = 42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

In [ ]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid,iid,verbose=True)

In [ ]:
def get_unseen_surprise(ratings, movies, userId):
  # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId']==userId['movieId'].tolist()]
  # 모든 영화의 movieId를 리스트로 생성
  total_movies = movies['movieId'].tolist()
  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수 : ',len(seen_movies),'추천 대상 영화 수 : ',len(unseen_movies),'전체 영화 수 : ',len(total_movies))
  return unseen_movies

unseen_movies = get_unseen_surprise(ratings,movies,9)

In [ ]:
# prediction 객체를 예측 평점이 높은 순으로 다시 정렬한 뒤 top n개의 객체에서 영화 아이디, 제목, 예측 평점 정보 반환하는 함수 생성

def recomm_movie_by_surprise(algo,userId,unseen_movies,top_n=10):

  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions = [algo.predict(str(userId),str(movieId)) for movieId in unseen_movies]

  # predictions list 객체는 surprise의 Predictions 객체를 원소로 가짐
  # 이를 est 값으로 정렬하기 위해 아래 함수 정의
  def sortkey_est(pred):
    return pred.est
  
  predictions.sort(key=sortkey_est,reverse=True)
  top_predictions = predictions[:top_n]

  # top_n으로 추출된 영화의 정보 추출, 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)['title']]

  top_movie_preds = [(id,title,rating) for id, title, rating in \
                     zip(top_movie_ids, top_movie_titles, top_movie_rating)]
  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings,movies,9)
top_movie_preds = recomm_movie_by_surprise(algo,9,unseen_movies,top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
  print(top_movie[1],':',top_movie[2])